## 1.ETL

In [1]:
import pandas as pd
qq = pd.read_csv('./Datasets/quora_question.csv')
qq.drop(columns=['test_id','question2'], inplace=True)
qq.shape

(2345796, 1)

In [2]:
random_sample = qq.sample(n=1000,random_state=6969) #try only 1000 samples
random_sample.shape

(1000, 1)

In [3]:
random_sample.head()

,question1
614123,Why won't China let Pope Francis visit?
795359,"Is it common to say ""you are welcome"" in when ..."
2209942,"Do G+ ""plus ones"" on posts actually do anythin..."
1383030,Can llp give loan to its partners?
529755,How many medals become won in Olympics ?


In [4]:
qq1000 = random_sample['question1'].values.tolist()

In [5]:
import nltk
# benepar.download('benepar_en3')
import benepar, spacy
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

def constituency_parser(text):
    doc = nlp(text)
    sent = list(doc.sents)[0]
    return  "(ROOT "+sent._.parse_string+")"

In [6]:
from tqdm import tqdm_notebook
train_data = list()
for idx in tqdm_notebook(range(len(qq1000))):
    train_data.append([qq1000[idx],constituency_parser(qq1000[idx])])

C:\Users\Guntsv\AppData\Local\Temp\ipykernel_24524\3282338791.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(qq1000))):


  0%|          | 0/1000 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Python3.10.4\lib\site-packages\torch\distributions\distribution.py:45: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [7]:
#Load data
train_data = pd.DataFrame(train_data)
train_data.rename(columns={0:'sentence',1:'parser'},inplace=True)
train_data.head()

,sentence,parser
0,Why won't China let Pope Francis visit?,(ROOT (SBARQ (WHADVP (WRB Why)) (SQ (MD wo) (R...
1,"Is it common to say ""you are welcome"" in when ...",(ROOT (SQ (VBZ Is) (NP (NP (PRP it))) (ADJP (J...
2,"Do G+ ""plus ones"" on posts actually do anythin...","(ROOT (SQ (VBP Do) (NP (NP (`` G+) (`` "") (CC ..."
3,Can llp give loan to its partners?,(ROOT (SQ (MD Can) (NP (NN llp)) (VP (VB give)...
4,How many medals become won in Olympics ?,(ROOT (SBARQ (WHNP (WHADJP (WRB How) (JJ many)...


In [8]:
def target(text):
    return  "<SOS> "+ text + " <EOS>"

train_data['target'] = train_data['sentence'].apply(target)

In [10]:
#Proprocessed Data
train_data.head()

,sentence,parser,target
0,Why won't China let Pope Francis visit?,(ROOT (SBARQ (WHADVP (WRB Why)) (SQ (MD wo) (R...,<SOS> Why won't China let Pope Francis visit? ...
1,"Is it common to say ""you are welcome"" in when ...",(ROOT (SQ (VBZ Is) (NP (NP (PRP it))) (ADJP (J...,"<SOS> Is it common to say ""you are welcome"" in..."
2,"Do G+ ""plus ones"" on posts actually do anythin...","(ROOT (SQ (VBP Do) (NP (NP (`` G+) (`` "") (CC ...","<SOS> Do G+ ""plus ones"" on posts actually do a..."
3,Can llp give loan to its partners?,(ROOT (SQ (MD Can) (NP (NN llp)) (VP (VB give)...,<SOS> Can llp give loan to its partners? <EOS>
4,How many medals become won in Olympics ?,(ROOT (SBARQ (WHNP (WHADJP (WRB How) (JJ many)...,<SOS> How many medals become won in Olympics ?...


In [12]:
def is_paren(tok):
    return tok == ")" or tok == "("

def deleaf(tree):
    nonleaves = ''
    for w in str(tree).replace('\n', '').split():
        w = w.replace('(', '( ').replace(')', ' )')
        nonleaves += w + ' '

    arr = nonleaves.split()
    for n, i in enumerate(arr):
        if n + 1 < len(arr):
            tok1 = arr[n]
            tok2 = arr[n + 1]
            if not is_paren(tok1) and not is_paren(tok2):
                arr[n + 1] = ""

    nonleaves = " ".join(arr)
    return nonleaves.split()

from nltk import ParentedTree

def Parsertokenize(synt_):
    synt_ = ParentedTree.fromstring(synt_)
    synt_ = deleaf(synt_)
    synt_ = [f'<{w}>' for w in synt_]
    return synt_

In [23]:
from subwordnmt.apply_bpe import BPE, read_vocabulary
import codecs

# load bpe codes
bpe_codes = codecs.open('./data/bpe.codes', encoding='utf-8')
bpe_vocab = codecs.open('./data/vocab.txt', encoding='utf-8')
bpe_vocab = read_vocabulary(bpe_vocab, 50)
bpe = BPE(bpe_codes, '@@', bpe_vocab, None)

def bpetokenize(sent_):
 # bpe segment and convert to tensor
    sent_ = bpe.segment(sent_).split()
    return sent_

In [27]:
train_load = pd.DataFrame()
train_load['sentence'] = train_data['sentence'].apply(bpetokenize)
train_load['parser'] = train_data['parser'].apply(Parsertokenize)
train_load['target'] = train_data['target']

In [28]:
train_load.head()

,sentence,parser,target
0,"[W@@, hy, won@@, 't, C@@, h@@, ina, let, P@@, ...","[<(>, <ROOT>, <(>, <SBARQ>, <(>, <WHADVP>, <(>...",<SOS> Why won't China let Pope Francis visit? ...
1,"[I@@, s, it, common, to, say, ""@@, you, are, w...","[<(>, <ROOT>, <(>, <SQ>, <(>, <VBZ>, <)>, <(>,...","<SOS> Is it common to say ""you are welcome"" in..."
2,"[D@@, o, G@@, +, ""@@, plus, on@@, es@@, "", on,...","[<(>, <ROOT>, <(>, <SQ>, <(>, <VBP>, <)>, <(>,...","<SOS> Do G+ ""plus ones"" on posts actually do a..."
3,"[C@@, an, ll@@, p, give, loan, to, its, partne...","[<(>, <ROOT>, <(>, <SQ>, <(>, <MD>, <)>, <(>, ...",<SOS> Can llp give loan to its partners? <EOS>
4,"[H@@, ow, many, medals, become, won, in, O@@, ...","[<(>, <ROOT>, <(>, <SBARQ>, <(>, <WHNP>, <(>, ...",<SOS> How many medals become won in Olympics ?...


In [62]:
import pickle
with open('./data/dictionary.pkl', 'rb') as f:
    vocab_transform = pickle.load(f)
vocab_dict = vocab_transform.word2idx

In [63]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('spacy',language='en_core_web_sm')
text_pipeline = lambda x: vocab_transform(tokenizer(x))

In [67]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

pad_idx = vocab_dict['<pad>'] ##get the pad index from the vocab

def collate_batch(batch):
    sent_list, synt_lst, trg_list = [], [], []
    for sen_, syn_, trg_ in batch:
        processed_sent = torch.tensor(text_pipeline(sen_), dtype=torch.int64)
        sent_list.append(processed_sent)
        processed_synt = torch.tensor(text_pipeline(syn_), dtype=torch.int64)
        synt_lst.append(processed_synt)
        processed_trg = torch.tensor(text_pipeline(trg_), dtype=torch.int64)
        trg_list.append(processed_trg)

    return pad_sequence(sent_list, padding_value=pad_idx, batch_first=True), pad_sequence(synt_lst, padding_value=pad_idx, batch_first=True), pad_sequence(trg_list, padding_value=pad_idx, batch_first=True)

In [18]:
# from torchtext.vocab import build_vocab_from_iterator
# from torchtext.data.utils import get_tokenizer

# tokenizer = get_tokenizer('spacy',language='en_core_web_sm')

# def yield_tokens(data_iter):
#     for text, _ in data_iter.itertuples(index=False):
#         yield tokenizer(text)
#     #loop through the data_iter, 
#     # Mind that the data_iter in this case is pandas Dataframe
#     # pass #remove this line and code here

# specials = ['<unk>','<pad>','<bos>','<eos>'] #create array of special tags for the vocab
# vocab_transform  = build_vocab_from_iterator(yield_tokens(train_data), specials = specials, special_first=True)

# #set_default_index of the vocab to unknown tag
# vocab_transform.set_default_index(vocab_transform["<unk>"]) #if you don't the id of this word, set it unk

In [24]:
# assert len(vocab_transform) == 3407 #only for 1000 samples

3407

In [23]:
# vocab_transform.get_itos()[:10]

['<unk>', '<pad>', '<bos>', '<eos>', '?', 'the', 'What', 'a', 'is', 'I']

In [8]:
# import pickle
# # # with open('vocab_transform_cnn.pickle', 'wb') as f:
# # #     pickle.dump(vocab_transform, f, protocol=pickle.HIGHEST_PROTOCOL)
# with open('vocab_transform_cnn.pickle', 'rb') as f:
#     vocab_transform = pickle.load(f)
# vocab_transform.get_itos()[:10]

['<unk>', '<pad>', '<bos>', '<eos>', '?', 'the', 'What', 'a', 'is', 'I']

In [6]:
# vocab_transform.idx2word

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: '<unk>',
 4: '<msk>',
 5: '<PAD>',
 6: '<ROOT>',
 7: '<ADJP>',
 8: '<-ADV>',
 9: '<ADVP>',
 10: '<-BNF>',
 11: '<CC>',
 12: '<CD>',
 13: '<-CLF>',
 14: '<-CLR>',
 15: '<CONJP>',
 16: '<-DIR>',
 17: '<DT>',
 18: '<-DTV>',
 19: '<EX>',
 20: '<-EXT>',
 21: '<FRAG>',
 22: '<FW>',
 23: '<-HLN>',
 24: '<IN>',
 25: '<INTJ>',
 26: '<JJ>',
 27: '<JJR>',
 28: '<JJS>',
 29: '<-LGS>',
 30: '<-LOC>',
 31: '<LS>',
 32: '<LST>',
 33: '<MD>',
 34: '<-MNR>',
 35: '<NAC>',
 36: '<NN>',
 37: '<NNS>',
 38: '<NNP>',
 39: '<NNPS>',
 40: '<-NOM>',
 41: '<NP>',
 42: '<NX>',
 43: '<PDT>',
 44: '<POS>',
 45: '<PP>',
 46: '<-PRD>',
 47: '<PRN>',
 48: '<PRP>',
 49: '<-PRP>',
 50: '<PRP$>',
 51: '<PRP-S>',
 52: '<PRT>',
 53: '<-PUT>',
 54: '<QP>',
 55: '<RB>',
 56: '<RBR>',
 57: '<RBS>',
 58: '<RP>',
 59: '<RRC>',
 60: '<S>',
 61: '<SBAR>',
 62: '<SBARQ>',
 63: '<-SBJ>',
 64: '<SINV>',
 65: '<SQ>',
 66: '<SYM>',
 67: '<-TMP>',
 68: '<TO>',
 69: '<-TPC>',
 70: '<-TTL>',
 7

In [27]:
# import torch

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(device)

# from torchtext.vocab import FastText
# fast_vectors = FastText(language='simple') ##Load fasttext with language = simple
# fast_embedding = fast_vectors.get_vecs_by_tokens(vocab_transform.get_itos()).to(device)

# #since the fasttext  has 300 embedding
# assert fast_embedding.shape == (len(vocab_transform), 300)

cpu


.vector_cache\wiki.simple.vec: 293MB [00:25, 11.4MB/s]                               
100%|██████████| 111051/111051 [00:11<00:00, 9527.14it/s] 


## 3. DataLoader

In [45]:
from torch.utils.data import DataLoader, Dataset

class DataWrap(Dataset):

    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __len__(self):
        return len(self.dataframe)
        
    def __getitem__(self, idx):
        return self.dataframe.iloc[idx]

In [49]:
from torch.utils.data import DataLoader
import torch
torch.manual_seed(6969)

train = DataWrap(train_load)
# val   = DataWrap(train_load.iloc[800:900] )
# test  = DataWrap(train_load.iloc[900:])
train_dataloader = DataLoader(train,batch_size=16, shuffle=True)
# test_dataloader = DataLoader(val,  batch_size=16)
# test_dataloader = DataLoader(test,batch_size=16)

In [55]:
# for i in train_dataloader:
#     print(i)
#     break

In [54]:
# from nltk import ParentedTree
# synt_ = '(ROOT (NP (NP (NN Finance)) (: :) (WHNP (WDT Which)) (SQ (VBZ is) (VP (VB be) (NP (DT an) (NNP Auroville) (NN resident)) (PP (IN for) (NP (DT a) (JJ long) (NN time))))) (. ?)))'
# synt_ = ParentedTree.fromstring(synt_)
# synt_ = deleaf(synt_)
# synt_ = [f'<{w}>' for w in synt_]
# synt_[:10]